<a href="https://colab.research.google.com/github/ar851060/palm_detection_prediction/blob/main/CCBDA_final_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re

def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

In [ ]:
import numpy as np
from tqdm.auto import tqdm

folder = "/content/drive/Shareddrives/CCBDA_final/allHands"
filenames = os.listdir(folder)

# print('Loading data ...')

data_root='/content/drive/Shareddrives/CCBDA_final/'
# file_array = np.zeros(24576)
# file_array = np.load('/content/drive/Shareddrives/CCBDA_final/Emb/001_1.npy')

# print(file_array.shape)
# print("-------------")

# for i in filenames:
#            file = np.load(folder + "/" + i)
#           #  print(file.shape)
#            file_array.append(file)
#           #  print(file_array.shape)

# #train = np.load(data_root + 'embbeding.npy')
# file_array = np.array(file_array)
train1 = np.load('/content/drive/Shareddrives/CCBDA_final/Autoencoder_total_emb/emb.npy', allow_pickle=True)
train2 = np.load('/content/drive/Shareddrives/CCBDA_final/embLine/embeddingLine.npy', allow_pickle=True)
# print(train1.shape)
# print(train2.shape)
train = np.hstack((train1,train2))
train_label = np.load('/content/drive/Shareddrives/CCBDA_final/label.npy', allow_pickle=True)
label = []
unlabel = []
filenames = sorted_alphanumeric(filenames)

for x, i in enumerate(filenames):
  if i[:-4] in train_label[:,0]:
    label.append(train[x,:])
  else:
    unlabel.append(train[x,:])

train = np.array(label)
unlabel = np.array(unlabel)

train_label = np.delete(train_label, (77,140,207,641), 0)
train_label = np.delete(train_label, (0,35), 1)

test = np.load(data_root + 'label.npy', allow_pickle=True)

print('Size of training data: {}'.format(train.shape))
print('Size of train_label data: {}'.format(train_label.shape))
# print('Size of label: {}'.format(label.shape))
print('Size of unlabel: {}'.format(unlabel.shape))

Size of training data: (847, 24608)
Size of train_label data: (847, 34)
Size of unlabel: (595, 24608)


In [ ]:
# def get_pseudo_labels(dataset, model, threshold=0.65):
#     # This functions generates pseudo-labels of a dataset using given model.
#     # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
#     # You are NOT allowed to use any models trained on external data for pseudo-labeling.
#     device = "cuda" if torch.cuda.is_available() else "cpu"

#     # Construct a data loader.
#     data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

#     # Make sure the model is in eval mode.
#     model.eval()
#     # Define softmax function.
#     softmax = nn.Softmax(dim=-1)

#     # Iterate over the dataset by batches.
#     for batch in tqdm(data_loader):
#         img, _ = batch

#         # Forward the data
#         # Using torch.no_grad() accelerates the forward process.
#         with torch.no_grad():
#             logits = model(img.to(device))

#         # Obtain the probability distributions by applying softmax on logits.
#         probs = softmax(logits)

#         # ---------- TODO ----------
#         # Filter the data and construct a new dataset.

#     # # Turn off the eval mode.
#     model.train()
#     return dataset

## Create Dataset

In [ ]:
VAL_RATIO = 0.1

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (762, 24608)
Size of validation set: (85, 24608)


rf: 0.19
extra: 0.17
knn: 0.32
ridge: 0.36

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier 


clf = RidgeClassifierCV(scoring = "f1_weighted", class_weight = "balanced", alphas = (0.005, 25e-5))
# clf = KNeighborsClassifier(n_neighbors=1, weights = "distance", n_jobs = -1,metric = "chebyshev")

clf.fit(train_x, train_y.astype(float))
pred = clf.predict(val_x)
f1_score(val_y.astype(float), pred, average = "weighted")

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.3592934983985806

In [ ]:
clf.alpha_

0.00025

In [ ]:
import pickle
filename = '/content/drive/Shareddrives/CCBDA_final/classifier.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
pred[0,:]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


## Create Model

In [ ]:

import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer0 = nn.Linear(96*16*16+32, 128)
        self.layer0_1 = nn.Linear(8192, 1024)
        self.layer1 = nn.Linear(2048, 1024)
        self.layer2 = nn.Linear(1024, 128)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 34) 

        self.act_fn = nn.Sigmoid()

    def forward(self, x):
        x = self.layer0(x)
        x = self.act_fn(x)

        # x = self.layer0_1(x)
        # x = self.act_fn(x)

        # x = self.layer1(x)
        # x = self.act_fn(x)

        # x = self.layer2(x)
        # x = self.act_fn(x)

        # x = self.layer3(x)
        # x = self.act_fn(x)

        x = self.out(x)
        
        return x

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from sklearn.metrics import f1_score
# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 100               # number of training epoch
learning_rate = 0.001       # learning rate
threshold = torch.tensor([0.3]).to(device)

# the path where checkpoint saved
model_path = '/content/drive/Shareddrives/CCBDA_final/classifier.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
# criterion = nn.BCELoss()
pos_weight = torch.ones([34]).to(device) * 10

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Define your optimization algorithm. 
# TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
# TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

DEVICE: cpu


In [ ]:
# start training
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device).type(torch.float)
        optimizer.zero_grad() 
        outputs = model(inputs)
        # outputs = torch.sigmoid(outputs)
        batch_loss = criterion(outputs, labels)
        train_pred = (outputs>threshold).float()
        # _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += f1_score(labels.cpu(), train_pred.cpu(), average='weighted') # (train_pred.cpu() == labels.cpu()).sum().item() / 34.0
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs.float(), labels.float())
                val_pred = (outputs>threshold).float() 
                # _, val_pred = torch.max(outputs, 1) 
            
                val_acc += f1_score(labels.cpu(), val_pred.cpu(), average='weighted') # (val_pred.cpu() == labels.cpu()).sum().item() / 34.0# get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-pack

[001/100] Train Acc: 0.002586 Loss: 1.730854 | Val Acc: 0.008841 loss: 1.348272
saving model with acc 0.009


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-pack

[002/100] Train Acc: 0.005740 Loss: 1.537901 | Val Acc: 0.010268 loss: 1.297004
saving model with acc 0.010


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-pack

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-pack

[099/100] Train Acc: 0.006945 Loss: 1.317073 | Val Acc: 0.011564 loss: 1.185161


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-pack

[100/100] Train Acc: 0.006944 Loss: 1.316343 | Val Acc: 0.011578 loss: 1.185425


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


## Testing

In [ ]:
# create testing dataset
test_set = TIMITDataset(unlabel, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        test_pred = (outputs>threshold).float()
        # _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

In [ ]:
predict

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 